In [3]:
# Taek / Oct 19
# Proof of concept for Berkeley Police Department Stop Data
import sys
import datascience
import folium

def file_to_dict(file_obj, delimeter):
  '''
  takes a data file object (delimetered) and returns the dataset as a list of dictionary
  '''
  line = file_obj.readline().strip()
  res = []
  keys = []
  if line != '':
    keys = line.split(delimeter)
  if len(keys) == 0:
    print('Invalid input (no column names found).')
    sys.exit(1)
  line = file_obj.readline().strip()
  cnt = 0
  while line != '':
    cnt += 1
    vals = line.split(delimeter)
    val_len = len(vals)
    if len(keys) != val_len:
      print('Invalid input (# of columns). - ' + str(cnt))
      sys.exit(1)
    record = {}
    for i in range(val_len):
      record[keys[i]] = vals[i].strip()
    res.append(record)
    line = file_obj.readline().strip()
  print('Read ' + str(len(res)) + ' records from file.')
  return res


In [4]:
f_name = 'data.txt'
f = open(f_name, 'r')
d = file_to_dict(f, '|')
t = datascience.Table.from_records(d)
t.show(10) # displaying ten rows

Read 8102 records from file.


address,date,dispositions,incident_number,incident_type,lat,long
"SAN PABLO AVE AND MARIN AVE, BERKELEY, CA",1/26/15 0:10,M,2015-00004825,Traffic,37.8867034,-122.2978679
"SAN PABLO AVE AND CHANNING WAY, BERKELEY, CA",1/26/15 0:50,M,2015-00004829,Traffic,37.862872,-122.2900545
"UNIVERSITY AVE AND NINTH ST, BERKELEY, CA",1/26/15 1:03,M,2015-00004831,Traffic,37.8687133,-122.2944035
"2000 BERKELEY WAY, BERKELEY, CA",1/26/15 7:16,BM4ICN,2015-00004848,Pedestrian,37.8734511,-122.2706227
"1700 SAN PABLO AVE, BERKELEY, CA",1/26/15 7:43,BM4ICN,2015-00004849,Pedestrian,37.8731719,-122.293804
"MARTIN LUTHER KING JR WAY AND UNIVERSITY AVE, BERKELEY, CA",1/26/15 9:46,OF4TCN,2015-00004865,Traffic,37.8716087,-122.2730269
"MARTIN LUTHER KING JR WAY AND UNIVERSITY AVE, BERKELEY, CA",1/26/15 10:05,OM4TCN,2015-00004870,Traffic,37.8716087,-122.2730269
"UNIVERSITY AVE AND MARTIN LUTHER KING JR WAY, BERKELEY, CA",1/26/15 10:21,OF2TCN,2015-00004876,Traffic,37.8716087,-122.2730269
"HASTE ST AND ELLSWORTH ST, BERKELEY, CA",1/26/15 10:49,OM2TCN,2015-00004882,Traffic,37.8654442,-122.2631606
"ADELINE ST AND OREGON ST, BERKELEY, CA",1/26/15 11:12,OM2TCN,2015-00004887,Traffic,37.857428,-122.2682845


In [5]:
# Playing with table-related methods in datascience package
t.stats()

statistic,address,date,dispositions,incident_number,incident_type,lat,long
min,", BERKELEY, CA",1/26/15 0:10,,2015-00004825,Bicycle,37.6222963,-122.0656859
max,"WOOLSEY ST/HARPER ST, BERKELEY, CA",8/9/15 7:27,WM4WWN,2015-00051386,Traffic,NA,NA
median,,,,,,,
sum,,,,,,,


In [6]:
# The summary above shows that there are missing values; min values with empty strings.
t.select(['incident_type', 'dispositions', 'address']) # Projection for selected columns

incident_type,dispositions,address
Traffic,M,"SAN PABLO AVE AND MARIN AVE, BERKELEY, CA"
Traffic,M,"SAN PABLO AVE AND CHANNING WAY, BERKELEY, CA"
Traffic,M,"UNIVERSITY AVE AND NINTH ST, BERKELEY, CA"
Pedestrian,BM4ICN,"2000 BERKELEY WAY, BERKELEY, CA"
Pedestrian,BM4ICN,"1700 SAN PABLO AVE, BERKELEY, CA"
Traffic,OF4TCN,"MARTIN LUTHER KING JR WAY AND UNIVERSITY AVE, BERKELEY, CA"
Traffic,OM4TCN,"MARTIN LUTHER KING JR WAY AND UNIVERSITY AVE, BERKELEY, CA"
Traffic,OF2TCN,"UNIVERSITY AVE AND MARTIN LUTHER KING JR WAY, BERKELEY, CA"
Traffic,OM2TCN,"HASTE ST AND ELLSWORTH ST, BERKELEY, CA"
Traffic,OM2TCN,"ADELINE ST AND OREGON ST, BERKELEY, CA"


In [7]:
t.sort('incident_number', True) # The second parameter means descending order. It creates a new table sorted.

address,date,dispositions,incident_number,incident_type,lat,long
"EASTSHORE HWY AND JONES ST, BERKELEY, CA",8/31/15 23:31,HM3TWN,2015-00051386,Traffic,NA,NA
"UNIVERSITY AVE AND MARTIN LUTHER KING JR WAY, BERKELEY, CA",8/31/15 22:11,BM2TCS,2015-00051375,Suspicious Vehicle,NA,NA
"TREMONT ST AND PRINCE ST, BERKELEY, CA",8/31/15 20:56,"BM2TWN, M",2015-00051356,Suspicious Vehicle,NA,NA
37.8609962420001~-122.256143699,8/31/15 19:38,M,2015-00051344,Pedestrian,37.86099624,-122.2561437
37.8609962420001~-122.256143699,8/31/15 15:02,WM4RAN,2015-00051299,Pedestrian,37.86099624,-122.2561437
"SIXTH ST AND BANCROFT WAY, BERKELEY, CA",8/31/15 15:01,WM4TWN,2015-00051298,Traffic,NA,NA
"2500 TELEGRAPH AVE, BERKELEY, CA",8/31/15 14:50,WM2RCN,2015-00051294,Pedestrian,NA,NA
"1400 SHATTUCK AVE, BERKELEY, CA",8/31/15 14:40,AM4IWN,2015-00051291,Pedestrian,NA,NA
37.8693028530001~-122.272234021,8/31/15 14:30,BM4RCN,2015-00051286,Pedestrian,37.86930285,-122.272234
"HEARST AVE AND MARTIN LUTHER KING JR WAY, BERKELEY, CA",8/31/15 13:22,WM4RWN,2015-00051270,Pedestrian,NA,NA


In [8]:
t.hist() # Plotting histograms for the columns
# Drawing capability requires further preprocessing; see the error message below

ValueError: The column 'address' contains non-numerical values. A histogram cannot be drawn for this table.

In [23]:
# Stop playing with Table-related methods in datascience package

NUM_SAMPLES = 1800
sub_table = t.take(range(1800)) # selecting the first 1800 rows because they have lat/long
sample_table = sub_table.sample([NUM_SAMPLES]) # randomly sampling 500 rows without replacement or weight

In [24]:
sample_table.show()

address,date,dispositions,incident_number,incident_type,lat,long
"2400 TELEGRAPH AVE, BERKELEY, CA",1/31/15 21:50,P,2015-00006169,Pedestrian,37.8674642,-122.2594219
"MARTIN LUTHER KING JR WAY AND PRINCE ST, BERKELEY, CA",2/10/15 7:07,HF2TCN,2015-00008266,Traffic,37.8524898,-122.2709328
"MARTIN LUTHER KING JR WAY AND CEDAR ST, BERKELEY, CA",2/11/15 14:05,WM2TWN,2015-00008598,Traffic,37.8778288,-122.2737079
"MARTIN LUTHER KING JR WAY AND ADELINE ST, BERKELEY, CA",2/20/15 18:42,M,2015-00010572,Traffic,37.8510652,-122.2705516
"MABEL ST AND HASKELL ST, BERKELEY, CA",2/18/15 3:03,"M, P",2015-00010034,Traffic,37.8511158,-122.2832476
"SHATTUCK AVE AND ADDISON ST, BERKELEY, CA",2/19/15 16:23,BM3TCN,2015-00010347,Traffic,37.871252,-122.2676888
"ASHBY AVE AND ADELINE ST, BERKELEY, CA",2/20/15 22:41,WM2TWN,2015-00010633,Traffic,37.8547666,-122.2689264
"SHATTUCK AVE AND DURANT AVE, BERKELEY, CA",3/8/15 18:59,BM3RCN,2015-00013925,Pedestrian,37.8666989,-122.2676075
"2200 OREGON ST, BERKELEY, CA",2/9/15 8:08,OM4TCN,2015-00008048,Traffic,37.8579794,-122.2639191
"DWIGHT WAY AND ELLSWORTH ST, BERKELEY, CA",1/30/15 1:43,HM4TWN,2015-00005715,Traffic,37.8645391,-122.2629878


In [31]:
# Mapping
berkeley_lat = 37.8700
berkeley_long = -122.2590
berkeley_map = folium.Map(location=[berkeley_lat, berkeley_long], zoom_start=14)

RADIUS = 30
RACES = ['A', 'B', 'H', 'O', 'W']
COLORS = ['#F21B3F', '#08BDBD', '#FF9914', '#F0F0F0', '#ABFF4F']

for row in sample_table.rows:
    popup_msg = row.date + '<br />' + row.incident_type + '<br />' + row.dispositions
#     berkeley_map.simple_marker([row.lat, row.long], popup=popup_msg) # Simple marker
    color = '#808080' # marker color by default
    if len(row.dispositions) >= 5:
        first_char = row.dispositions[0]
        if first_char in RACES:
            race_index = RACES.index(first_char)
            color = COLORS[race_index]
        else:
#         Incident disposition wasn't completely reported.
            continue
    else:
#         Incident disposition wasn't completely reported.
        continue
    berkeley_map.circle_marker([row.lat, row.long],
                               radius=RADIUS,
                               line_color=color,
                               fill_color=color,
                               popup=popup_msg) # Colored marker
berkeley_map.create_map(path='berkeley_stop_map.html')